In [1]:
from itertools import cycle
from functools import cache

In [2]:
class Player():
    def __init__(self, name, starting):
        self.name = name
        self.position = starting
        self.score = 0
        self.rolls = 0
    
    def roll(self, die):
        forward = next(die) + next(die) + next(die)
        self.position = (self.position + forward - 1) % 10 + 1
        self.score += self.position
        self.rolls += 3

def deterministic_die(n):
    yield from cycle(range(1, n+1))

In [3]:
def play(pos1, pos2):
    p1 = Player("1", starting=pos1)
    p2 = Player("2", starting=pos2)
    die = deterministic_die(100)

    while True:
        p1.roll(die)
        if p1.score >= 1000:
            return p1, p2
        p2.roll(die)
        if p2.score >= 1000:
            return p2, p1

In [4]:
winner, loser = play(pos1=5, pos2=9)
print(f"Player {winner.name} wins")
print(loser.score * (winner.rolls + loser.rolls))

Player 2 wins
989352


In [5]:
roll_possibilities = {3:1, 4:3, 5:6, 6:7, 7:6, 8:3, 9:1}

@cache
def assess(pos1, pos2, score1=0, score2=0, p1_is_next=True):
    wins  = [0, 0]
    pos   =   pos1 if p1_is_next else pos2
    score = score1 if p1_is_next else score2

    for roll, combinations in roll_possibilities.items():
        next_pos = (pos + roll - 1) % 10 + 1
        next_score = score + next_pos
        if next_score >= 21:
            wins[0 if p1_is_next else 1] += combinations
        else:
            if p1_is_next:
                p1_wins, p2_wins = assess(next_pos, pos2, next_score, score2, p1_is_next=False)
            else:
                p1_wins, p2_wins = assess(pos1, next_pos, score1, next_score, p1_is_next=True)
            wins[0] += p1_wins * combinations
            wins[1] += p2_wins * combinations
    return wins

p1_wins, p2_wins = assess(pos1=5, pos2=9)
print(f"Player 1 wins in {p1_wins} universes")
print(f"Player 2 wins in {p2_wins} universes")

Player 1 wins in 430229563871565 universes
Player 2 wins in 370143448743170 universes
